Notebook Corresponding to sub section 4 

# Making Recommendations Based on Popularity

These datasets are hosted on: https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data

They were originally published by: Blanca Vargas-Govea, Juan Gabriel González-Serna, Rafael Ponce-Medellín. Effects of relevant contextual features in the performance of a restaurant recommender system. In RecSys11: Workshop on Context Aware Recommender Systems (CARS-2011), Chicago, IL, USA, October 23, 2011.

## Restaurants data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# rating_final.csv
url = 'https://drive.google.com/file/d/1ptu4AlEXO4qQ8GytxKHoeuS1y4l_zWkC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
frame = pd.read_csv(path)

# chefmozcuisine.csv
url = 'https://drive.google.com/file/d/1S0_EGSRERIkSKW4D8xHPGZMqvlhuUzp1/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
cuisine = pd.read_csv(path)

# 'geoplaces2.csv'
url = 'https://drive.google.com/file/d/1ee3ib7LqGsMUksY68SD9yBItRvTFELxo/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
geodata = pd.read_csv(path, encoding = 'CP1252') # change encoding to 'mbcs' in Windows

On the "frame" dataset we have the ratings users have given to places. Ratings go from 0 to 2.

In [ ]:
frame.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In the `geodata` dataset we have info about the places. We will only use the `name` column.

In [ ]:
geodata.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [ ]:
places =  geodata[['placeID', 'name']]
places.head()

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rincón de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata


In the `cuisine` dataset we have the type of cuisine that restaurants offer.

In [ ]:
cuisine.head(3)

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American


## Popularity/Quality based recommmender system

Let's group places by rating, and look at their average rating. This is an **explicit** rating given by users.

In [ ]:
rating = pd.DataFrame(frame.groupby('placeID')['rating'].mean())
rating.sort_values("rating", ascending=False).head()

,rating
placeID,
132955,2.000000
135034,2.000000
134986,2.000000
132922,1.833333
132755,1.800000


The top rated places have a perfect score of 2/2. But how many reviews do these places have?

In [ ]:
frame.query("placeID==132955")

,userID,placeID,rating,food_rating,service_rating
934,U1004,132955,2,2,2
960,U1061,132955,2,2,2
996,U1059,132955,2,1,2
1014,U1097,132955,2,2,1
1080,U1096,132955,2,2,2


Looks like only 5 people went to this place. Maybe they're just the owner's friends! Or maybe they're really top-quality places, but too niche to recommend to the masses.

We can also look at how many times each restaurant has received a rating. The ratings count is an **implicit** rating.

In [ ]:
rating['rating_count'] = frame.groupby('placeID')['rating'].count()
rating.sort_values("rating_count", ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


Some places have been visited around 30 times. They are more popular than the top rated places, but received lower explicit ratings.

Let's locate the most popular place, and get some info about it:

In [ ]:
rating.sort_values('rating_count', ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [ ]:
# placeId of most popular place
top_popular_placeID = rating.sort_values('rating_count', ascending=False).head(1).index[0]

# name of the most popular place
places[places['placeID']==top_popular_placeID]

,placeID,name
121,135085,Tortas Locas Hipocampo


In [ ]:
# cuisine of the most popular place
cuisine[cuisine['placeID']==top_popular_placeID]

,placeID,Rcuisine
44,135085,Fast_Food


The most popular place is "Tortas Locas Hipocampo", a fast food place that has received 36 reviews and it has an average score of 1.33.

### Challenge:

Find a hybrid system to sort restaurants, so that you can recommend the "best" places: restaurants that are both high rated and popular.

In [ ]:
popular = pd.DataFrame(frame.groupby('placeID')['rating'].agg(["mean", "count"]).nlargest(100,"count"))
popular = popular[(popular['mean']>1.4)].head(10)
popular

In [ ]:
popular.merge(places, how = 'left', on = 'placeID')

,placeID,mean,count,name
0,135025,1.666667,15,El Rincon de San Francisco
1,135028,1.533333,15,La Virreina
2,135051,1.428571,14,Restaurante Versalles
3,132754,1.461538,13,Cabana Huasteca
4,135045,1.461538,13,Restaurante la Gran Via
5,135075,1.692308,13,Mariscos El Pescador
6,132723,1.416667,12,Gordas de morales
7,135030,1.583333,12,Preambulo Wifi Zone Cafe
8,135066,1.500000,12,Restaurante Guerra
9,132768,1.600000,10,Mariscos Tia Licha
